In [1]:
from pythonosc.udp_client import SimpleUDPClient

import pandas as pd
import numpy as np
from time import sleep

In [2]:
# read the osc data and check min/max
df_score = pd.read_csv('data\osc_data.csv')
df_score[[col for col in df_score.columns if '_ts' in col]].describe()

,low_pitch_val_ts12,low_pitch_val_ts16,high_pitch_val_ts12,high_pitch_val_ts16,synth_val_ts
count,56.000000,56.000000,56.000000,56.000000,56.000000
mean,8.000000,9.821429,6.803571,8.178571,4.642857
std,2.215647,3.379656,2.726434,3.968545,1.518714
min,5.000000,5.000000,4.000000,4.000000,1.000000
25%,6.000000,7.000000,4.000000,4.000000,4.000000
50%,8.000000,10.000000,6.000000,7.500000,5.000000
75%,9.000000,12.000000,9.000000,11.000000,6.000000
max,12.000000,16.000000,13.000000,17.000000,7.000000


In [8]:
# make a variable to send OSC messages
osc_power = SimpleUDPClient("127.0.0.1", 8000)
osc_messenger = SimpleUDPClient("127.0.0.1", 8001)

# update the changes with change in cycles
cycle_change = [0,8,30,36,44]
metro_change = [140,100,110,160,120]   
change = 0
bpm = 120


# loop for all the cycles
for i in range(df_score.shape[0]):

    # get current cycle info
    info = df_score.iloc[i]

    #check if metro needs to be updated
    if (change<len(cycle_change) and i == cycle_change[change]):
        bpm = metro_change[change]
        change = change + 1
        osc_power.send_message('/metro', bpm)
    
    # send needed info to the Pd patch
    osc_messenger.send_message('/drum_hi_ts', info['high_pitch_val_ts16'])
    osc_messenger.send_message('/drum_low_ts', info['low_pitch_val_ts16'])
    osc_messenger.send_message('/groove_seed', change)

    # reset the bpm button
    osc_messenger.send_message('/set_bpm', 1)

    # start the BPM if its first cycle
    if i == 0:
        osc_power.send_message('/power',1)

    # add a delay according to the metro for updated values
    sleep(int(8*60/bpm))

# end the BPM on pureData
osc_power.send_message('/power',0)

In [4]:
import time

start = (time.time())

# update the changes with change in cycles
cycle_change = [0,8,30,36,44]
metro_change = [140,100,110,160,120]   
change = 0
bpm = 120


# loop for all the cycles
for i in range(df_score.shape[0]):

    #check if metro needs to be updated
    if (change<len(cycle_change) and i == cycle_change[change]):
        print(change, time.time()-start)
        bpm = metro_change[change]
        change = change + 1

    # add a delay according to the metro for updated values
    sleep(int(8*60/bpm))

0 0.0004963874816894531
1 24.0046443939209
2 112.0149040222168
3 136.0243308544159
4 160.10480499267578
